复现代码
2、比较使用高级API与使用低级API模型性能的差别，分析可能造成的原因

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
with tf.Graph().as_default() as g:
    #输入、标记占位符
    inputs = tf.placeholder(shape=[None,784],dtype=tf.float32)
    labels = tf.placeholder(shape=[None,10],dtype=tf.float32)
    
    #创建128个隐藏层神经元参数
    hidden_weight = tf.Variable(tf.random_normal([784,128]),name='hidden_weight')
    hidden_bias = tf.Variable(tf.zeros([128, ]),name='hidden_bias')
    
    #隐藏层前向传播
    hidden_output = tf.nn.relu(tf.matmul(inputs,hidden_weight) + hidden_bias)
    
    #创建输出层10个神经元参数
    output_weight = tf.Variable(tf.random_normal([128,10],name='output_weight'))
    output_bias = tf.Variable(tf.zeros([10, ]),name='output_bias')
    
    #输出层前向传播
    logits = tf.matmul(hidden_output,output_weight) + output_bias
    output = tf.nn.softmax(logits)
    
    #代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=labels))
    
    #正确率
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(labels,axis=1),tf.argmax(output,axis=1)),tf.float32))
    
    
    with tf.Session(graph=g) as sess:
        #定义梯度下降法优化器
        optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optim.minimize(loss)
        sess.run(tf.global_variables_initializer())
        
        #训练模型
        for step in range(20000):
            batch_images,batch_labels = mnist.train.next_batch(32)
            res_loss,_ = sess.run([loss,train_op],feed_dict={inputs:batch_images,labels:batch_labels})
            
            #输出代价并验证模型：
            if step % 500 == 0:
                accs = []
                
                for test_step in range(10000 // 32):
                    batch_images,batch_labels = mnist.test.next_batch(32)
                    res_acc = sess.run(acc,feed_dict={inputs:batch_images,labels:batch_labels})
                    accs.append(res_acc)
                accs = np.mean(accs)
                print('step %5d,loss %2.4f,acc %.4f' % (step,res_loss,accs))


step     0,loss 173.6963,acc 0.0483
step   500,loss 10.5830,acc 0.7551
step  1000,loss 2.7155,acc 0.8150
step  1500,loss 1.1467,acc 0.8399
step  2000,loss 2.3002,acc 0.8494
step  2500,loss 3.6629,acc 0.8558
step  3000,loss 0.0417,acc 0.8673
step  3500,loss 6.4831,acc 0.8687
step  4000,loss 1.4581,acc 0.8714
step  4500,loss 0.7100,acc 0.8752
step  5000,loss 1.7195,acc 0.8812
step  5500,loss 3.0660,acc 0.8833
step  6000,loss 1.3766,acc 0.8869
step  6500,loss 0.1128,acc 0.8866
step  7000,loss 2.1298,acc 0.8892
step  7500,loss 2.2534,acc 0.8928
step  8000,loss 3.2966,acc 0.8902
step  8500,loss 0.4713,acc 0.8905
step  9000,loss 2.3015,acc 0.8940
step  9500,loss 0.5354,acc 0.8964
step 10000,loss 0.6774,acc 0.8919
step 10500,loss 2.3059,acc 0.8965
step 11000,loss 1.5199,acc 0.8988
step 11500,loss 0.3335,acc 0.8947
step 12000,loss 0.2331,acc 0.9003
step 12500,loss 1.1859,acc 0.8956
step 13000,loss 0.0273,acc 0.9005
step 13500,loss 0.4404,acc 0.9001
step 14000,loss 1.1633,acc 0.9015
step 14500,

高级API是为了降低大家的编码难度而设置的。Tensorflow更高层的API使得配置，训练，评估多种多样的机器学习模型更简单方便了。